Файн-тюнинг больших языковых моделей

In [ ]:
pip install --upgrade torch transformers peft bitsandbytes accelerate

In [ ]:
pip install huggingface_hub transformers bitsandbytes accelerate peft trl datasets mistral-common scipy

In [ ]:
from huggingface_hub import login
login()

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, default_data_collator
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
import os
import glob
import re
from pathlib import Path

In [ ]:
model_id = "google/gemma-3-1b-it"

bnb_config = BitsAndBytesConfig(
	load_in_4bit=True,
	bnb_4bit_quant_type="nf4",
	bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
	model_id,
	quantization_config=bnb_config,
    device_map={"": "cuda:0"} if torch.cuda.is_available() else {"": "cpu"}
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [ ]:
file_path = 'https://github.com/Vanya119/mdl/raw/refs/heads/main/3colab.jsonl'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

ds = load_dataset("json", data_files=file_path, split="train")

In [ ]:
from peft import PeftModel

if isinstance(model, PeftModel):
    model = model.unload()

training_args = TrainingArguments(
    output_dir="./gemma-finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-6,
    num_train_epochs=1,
    logging_steps=10,
    save_strategy="epoch",
    fp16=False,
    bf16=False,
    report_to="none"
)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)
trainer = SFTTrainer(
	model=model,
	args=training_args,
	train_dataset=ds,
	peft_config=lora_config,
	processing_class=tokenizer,
)

trainer.train()

print("Обучение успешно завершено!")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'pad_token_id': 1}.


Step,Training Loss


Обучение успешно завершено!


In [ ]:
base_model_id = "google/gemma-3-1b-it"
output_dir = "./gemma-finetuned"
merged_model_dir = "./gemma-finetuned-merged"
all_checkpoints = glob.glob(os.path.join(output_dir, 'checkpoint-*'))
if not all_checkpoints:
	raise FileNotFoundError(f"Не найдены папки 'checkpoint-' в каталоге {output_dir}. Убедитесь, что обучение завершилось.")
def extract_step(checkpoint_path):
	match = re.search(r'checkpoint-(\d+)', checkpoint_path)
	return int(match.group(1)) if match else -1
latest_checkpoint = max(all_checkpoints, key=extract_step)
lora_adapter_dir = latest_checkpoint
print(f"последний чекпоинт: {lora_adapter_dir}")

последний чекпоинт: ./gemma-finetuned/checkpoint-8


In [ ]:
print("Загрузка базовой модели...")
base_model = AutoModelForCausalLM.from_pretrained(
	base_model_id,
	dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
	device_map={"": "cuda:0"} if torch.cuda.is_available() else {"": "cpu"}
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
print("Слияние LoRA-адаптера с базовой моделью...")
model = PeftModel.from_pretrained(base_model, lora_adapter_dir)
merged_model = model.merge_and_unload()
os.makedirs(merged_model_dir, exist_ok=True)
merged_model.save_pretrained(merged_model_dir, safe_serialization=True)
tokenizer.save_pretrained(merged_model_dir)
print('Готово')
print(f"Файлы модели находятся в: {merged_model_dir}")

Загрузка базовой модели...
Слияние LoRA-адаптера с базовой моделью...
Готово
Файлы модели находятся в: ./gemma-finetuned-merged


In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git

Cloning into 'llama.cpp'...
remote: Enumerating objects: 72021, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 72021 (delta 79), reused 34 (delta 34), pack-reused 71882 (from 2)
Receiving objects: 100% (72021/72021), 237.73 MiB | 35.50 MiB/s, done.
Resolving deltas: 100% (52072/52072), done.


In [ ]:
merged_model_dir = Path.cwd() / Path("./gemma-finetuned-merged")
HF_MODEL_PATH_STR = str(merged_model_dir)
GGUF_MODEL_PATH_STR = str(merged_model_dir / "gemma-finetuned-temp.gguf")
CONVERT_SCRIPT_PATH = "./llama.cpp/convert_hf_to_gguf.py"

!python {CONVERT_SCRIPT_PATH} "{HF_MODEL_PATH_STR}" --outtype f16 --outfile "{GGUF_MODEL_PATH_STR}"

INFO:hf-to-gguf:Loading model: gemma-finetuned-merged
INFO:hf-to-gguf:Model architecture: Gemma3ForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,                 torch.bfloat16 --> F16, shape = {1152, 262144}
INFO:hf-to-gguf:blk.0.attn_norm.weight,            torch.bfloat16 --> F32, shape = {1152}
INFO:hf-to-gguf:blk.0.ffn_down.weight,             torch.bfloat16 --> F16, shape = {6912, 1152}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,             torch.bfloat16 --> F16, shape = {1152, 6912}
INFO:hf-to-gguf:blk.0.ffn_up.weight,               torch.bfloat16 --> F16, shape = {1152, 6912}
INFO:hf-to-gguf:blk.0.post_attention_norm.weight,  torch.bfloat16 --> F32, shape = {1152}
INFO:hf-to-gguf:blk.0.post_ffw_norm.weight,        torch.bfloat16 --> F32, shape = {1152}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,             torch.bfloat16 --> 